In [1]:
def fxses(date, security):
    x = dt.datetime.timestamp(dt.datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S'))
    element = dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d')
    y = dt.datetime.strptime(element, '%Y-%m-%d')
    y_ts = dt.datetime.timestamp(y)

    # above codes, capture and change current time to timestamp
    # Convert the timestamp into YYYY-MM-DD format, in a way stripping the HH:MM:SS elements
    # capture the timestamp of the stripped date data, it would arrive YYYY-MM-DD, 00:00:00
    
    if y.weekday() == 6: # if the date was a Sunday, refer 2 days back, which is Friday
        tokyo_0 = y_ts - 172800 - 3600
        tokyo_1 = y_ts - 172800 + 28800
        london_0 = y_ts - 172800 + 25200
        london_1 = y_ts - 172800 + 57600
        newyork_0 = y_ts - 172800 + 43200
        newyork_1 = y_ts - 172800 + 72000
    elif y.weekday() == 0: # if the date was a Monday, refer 3 days back, which is Friday
        tokyo_0 = y_ts - 259200 - 3600
        tokyo_1 = y_ts - 259200 + 28800
        london_0 = y_ts - 259200 + 25200
        london_1 = y_ts - 259200 + 57600
        newyork_0 = y_ts - 259200 + 43200
        newyork_1 = y_ts - 259200 + 72000
    else: 
        tokyo_0 = y_ts - 86400 - 3600
        tokyo_1 = y_ts - 86400 + 28800
        london_0 = y_ts - 86400 + 25200
        london_1 = y_ts - 86400 + 57600
        newyork_0 = y_ts - 86400 + 43200
        newyork_1 = y_ts - 86400 + 72000
        
    # above using timestamp to segment out the timestamp for the respective hours of the previous day
    # value of 1 represent 1 second, so convert to hours accordingly
    
    tokyo_b = dt.datetime.fromtimestamp(tokyo_0)
    tokyo_e = dt.datetime.fromtimestamp(tokyo_1)
    london_b = dt.datetime.fromtimestamp(london_0)
    london_e = dt.datetime.fromtimestamp(london_1)
    newyork_b = dt.datetime.fromtimestamp(newyork_0)
    newyork_e = dt.datetime.fromtimestamp(newyork_1)
    
    # above convert the timestamp to datetime value
    
    tokyo_high = markets.loc[tokyo_b:tokyo_e]['MID_HIGH'].max()
    tokyo_low = markets.loc[tokyo_b:tokyo_e]['MID_LOW'].min()
    london_high = markets.loc[london_b:london_e]['MID_HIGH'].max()
    london_low = markets.loc[london_b:london_e]['MID_LOW'].min()
    newyork_high = markets.loc[newyork_b:newyork_e]['MID_HIGH'].max()
    newyork_low = markets.loc[newyork_b:newyork_e]['MID_LOW'].min()
    
    # above using reuters API to capture the high & low in hourly interval
    
    if newyork_high > london_high > tokyo_high and newyork_low > london_low > tokyo_low:
        trade = True
        direction = 'long'
        return trade, direction, london_high, london_low
    elif newyork_high < london_high < tokyo_high and newyork_low < london_low < tokyo_low:
        trade = True
        direction = 'short'
        return trade, direction, london_high, london_low
    else: 
        trade = False
        direction = 'N/A'
        return trade, direction, london_high, london_low
    
    # above is the trading rule, the key in this whole exercise
    # target to return trade signal, if neither criteria fulfilled, return False
    # False indicating no trade
    

In [2]:
def sameday(i):
    
    # converting the timestamp to datetime object and extract the date values
    x = dt.datetime.strptime(str(markets.index[i-1]), '%Y-%m-%d %H:%M:%S').date()
    y = dt.datetime.strptime(str(markets.index[i]), '%Y-%m-%d %H:%M:%S').date()
    
    # return true if it is the same date
    return (x == y)

In [3]:
import refinitiv.data as rd
import datetime as dt
import pandas as pd

security = 'GBPNZD='
date_0 = '2023-02-01'
date_1 = '2023-12-01'

rd.open_session()
markets = rd.get_history(universe=[security], fields=['MID_HIGH','MID_LOW'],
                     interval="1h", start=date_0, end=date_1
                    )
# input parameter to generate the market data

In [4]:
# started off with no trade
entry = 0
SL = 0 
TP = 0
pnl = []
trade_entry = {}

for i in range(24, len(markets)): 
    if entry == 0: # if no trade
        trade, direction, high, low = fxses(markets.index[i], security)
        if sameday(i) == False:
            if dt.datetime.strptime(str(markets.index[i]), '%Y-%m-%d %H:%M:%S').date().weekday() == 6: # a brute force way to resolve issue on weekend
                continue
            elif trade == False:
                continue
            elif direction == 'long':
                entry = markets['MID_HIGH'][i]
                SL = low
                TP = entry + (entry - SL)
                trade_entry[markets.index[i]] = entry
            elif direction == 'short':
                entry = -markets['MID_LOW'][i]
                SL = high
                TP = -entry + (-entry - SL)
                trade_entry[markets.index[i]] = entry
        else: continue
    elif entry > 0: # if long
        if markets['MID_HIGH'][i] >= TP:
            pnl.append(markets['MID_HIGH'][i] - entry)
            entry = 0
            SL = 0
            TP = 0
            continue
        elif markets['MID_LOW'][i] <= SL:
            pnl.append(markets['MID_LOW'][i] - entry)
            entry = 0
            SL = 0
            TP = 0
            continue
        else: continue
    elif entry < 0: # if short
        if markets['MID_HIGH'][i] >= SL:
            pnl.append(-entry - markets['MID_HIGH'][i])
            entry = 0
            SL = 0
            TP = 0
            continue
        elif markets['MID_LOW'][i] <= TP:
            pnl.append(-entry - markets['MID_LOW'][i])
            entry = 0
            SL = 0
            TP = 0
            continue
        else: continue
    else: continue
        

In [5]:
display(trade_entry)

{Timestamp('2023-02-13 00:00:00'): -1.9092,
 Timestamp('2023-02-20 00:00:00'): 1.9299,
 Timestamp('2023-02-22 00:00:00'): 1.9522,
 Timestamp('2023-03-06 00:00:00'): 1.9374,
 Timestamp('2023-03-10 00:00:00'): 1.955,
 Timestamp('2023-03-27 00:00:00'): 1.9739,
 Timestamp('2023-03-29 00:00:00'): -1.969,
 Timestamp('2023-04-12 00:00:00'): 2.0077,
 Timestamp('2023-04-14 00:00:00'): -1.9845,
 Timestamp('2023-04-27 00:00:00'): 2.0392,
 Timestamp('2023-05-04 00:00:00'): 2.0248,
 Timestamp('2023-05-05 00:00:00'): -1.999,
 Timestamp('2023-05-23 00:00:00'): -1.9747,
 Timestamp('2023-05-29 00:00:00'): 2.0394,
 Timestamp('2023-06-08 00:00:00'): 2.05995,
 Timestamp('2023-06-13 00:00:00'): -2.0422,
 Timestamp('2023-07-06 00:00:00'): 2.0568,
 Timestamp('2023-07-24 00:00:00'): 2.085,
 Timestamp('2023-07-27 00:00:00'): 2.0818,
 Timestamp('2023-08-01 00:00:00'): -2.0643,
 Timestamp('2023-08-16 00:00:00'): 2.1376,
 Timestamp('2023-09-05 00:00:00'): 2.12825,
 Timestamp('2023-10-10 00:00:00'): -2.0294,
 Time

In [6]:
display(pnl)

[0.008799999999999919,
 0.01100000000000012,
 -0.030000000000000027,
 0.017800000000000038,
 0.020999999999999908,
 0.009500000000000064,
 -0.01089999999999991,
 -0.018699999999999717,
 -0.0250999999999999,
 -0.013600000000000279,
 -0.02110000000000012,
 0.017400000000000082,
 -0.011300000000000088,
 0.011400000000000077,
 -0.018450000000000077,
 -0.01695000000000002,
 -0.008799999999999919,
 -0.017199999999999882,
 -0.012299999999999756,
 -0.015500000000000291,
 0.01540000000000008,
 0.011150000000000215,
 -0.015600000000000058,
 0.020299999999999763,
 -0.009300000000000086,
 -0.020199999999999996,
 -0.021999999999999797,
 -0.013599999999999834,
 -0.01839999999999975,
 0.018100000000000005]

In [7]:
total_pnl = sum(pnl)
No_trades = len(trade_entry)


print(f'accumulated {No_trades} trades with the profit and loss of {total_pnl}')

accumulated 30 trades with the profit and loss of -0.15714999999999923
